# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df= pd.read_csv(
    "../resources/city_weather.csv", dtype="object", encoding="utf-8")

# Visualize
weather_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed
0,Yellowknife,62.46,-114.35,8.6,84.0,90.0,3.36
1,Port Elizabeth,-33.92,25.57,59.0,93.0,0.0,4.7
2,Qaanaaq,77.48,-69.36,-11.31,68.0,2.0,9.69
3,Mataura,-46.19,168.86,55.0,60.0,0.0,10.0
4,Deputatsky,69.3,139.9,-5.17,92.0,100.0,3.8
...,...,...,...,...,...,...,...
540,Berbera,10.44,45.01,77.47,73.0,52.0,4.29
541,Tommot,58.96,126.29,4.21,89.0,100.0,8.32
542,Rio Grande City,26.38,-98.82,73.0,87.0,0.0,4.0
543,Rockland,41.17,-74.05,59.0,60.0,4.0,4.65


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
weather_df.columns

Index(['City', 'Lat', 'Long', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed'],
      dtype='object')

In [4]:
weather_df.dtypes

City          object
Lat           object
Long          object
Max Temp      object
Humidity      object
Cloudiness    object
Wind Speed    object
dtype: object

In [7]:
location = weather_df[["Lat", "Long"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

In [8]:
gmaps.configure(api_key=g_key)

In [12]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights = humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
plt.savefig("Heat_Map.png")

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
weather_df.replace("", "NaN", inplace = True)
weather_df.dropna(subset = ["City"], inplace = True)
weather_df.dropna(subset = ["Cloudiness"], inplace = True)
weather_df.dropna(subset = ["Humidity"], inplace = True)
weather_df.dropna(subset = ["Lat"], inplace = True)
weather_df.dropna(subset = ["Long"], inplace = True)
weather_df.dropna(subset = ["Max Temp"], inplace = True)
weather_df.dropna(subset = ["Wind Speed"], inplace = True)
weather_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed
0,Yellowknife,62.46,-114.35,8.6,84.0,90.0,3.36
1,Port Elizabeth,-33.92,25.57,59.0,93.0,0.0,4.7
2,Qaanaaq,77.48,-69.36,-11.31,68.0,2.0,9.69
3,Mataura,-46.19,168.86,55.0,60.0,0.0,10.0
4,Deputatsky,69.3,139.9,-5.17,92.0,100.0,3.8
...,...,...,...,...,...,...,...
540,Berbera,10.44,45.01,77.47,73.0,52.0,4.29
541,Tommot,58.96,126.29,4.21,89.0,100.0,8.32
542,Rio Grande City,26.38,-98.82,73.0,87.0,0.0,4.0
543,Rockland,41.17,-74.05,59.0,60.0,4.0,4.65


In [15]:
vacation_df = weather_df[weather_df["Max Temp"] > "70"]
vacation_df = vacation_df[vacation_df["Max Temp"] < "80"]
vacation_df = vacation_df[vacation_df["Wind Speed"] < "10"]
vacation_df = vacation_df[vacation_df["Cloudiness"] == "0.0"]
vacation_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed
188,Kibala,9.11,18.35,71.6,83.0,0.0,1.79
225,Nizwá,22.93,57.53,74.46,54.0,0.0,0.78
249,Wagar,16.15,36.2,71.42,72.0,0.0,1.81
331,Hai’an,32.54,120.45,72.0,32.0,0.0,1.01
378,Boffa,10.17,-14.03,77.31,92.0,0.0,1.63
417,Kalabagh,32.97,71.56,75.81,18.0,0.0,1.95
463,Topolobampo,25.6,-109.05,73.99,89.0,0.0,1.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
maps_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

KeyError: 'Passing list-likes to .loc or [] with any missing labels is no longer supported, see https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike'

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
